In [83]:
import pandas as pd
import numpy as np

In [84]:

#SAME CODE AS WEEK 2 !



filename = "../.gitignore/data_a (Florent).csv"
df = pd.read_csv(filename, skipinitialspace=True) #Open the csv file with no space before and after the values

#Part 1 : cleaning the data

#A. Drop the duplicates

nb_of_duplicates = len(df)-len(df.drop_duplicates()) #How many duplicates do I have ?

if nb_of_duplicates != 0 :
    df.drop_duplicates(inplace=True)

#B. Empty cells :

df.isnull().sum() #How many empty cells are there for each column

cleaned_df = df.dropna(subset=['Price'])#Delete the 3992 empty price
cleaned_df = cleaned_df.dropna(subset=['Number of rooms'])#Delete the 1918 empty Number of rooms
cleaned_df = cleaned_df.dropna(subset=['Living Area'])#Delete the 11677 empty Living Area (60605 left)

cleaned_df["Garden"].fillna(False, inplace = True) #Replace NaN value by False in the Garden Column
cleaned_df["Furnished"].fillna(False, inplace = True) #Replace NaN value by False in the Furnished Column
cleaned_df["Fully equipped kitchen"].fillna(False, inplace = True) #Replace NaN value by False in the Furnished Column

del cleaned_df['Area of the terrace'] #Delete the column Area of the terrace
del cleaned_df['Area of the garden'] #Delete the column Area of the garden
del cleaned_df['State of the building'] #Delete the column State of the building
del cleaned_df['Surface area of the plot of land'] #Delete the column Surface area of the plot of land
del cleaned_df['Surface of the land'] #Delete the column Surface area of land
del cleaned_df['Number of facades'] #Delete the column Number of facades

nb_null_values = cleaned_df.isnull().sum()

#C. No blank spaces 
# Already done in the opening of the csv file

#D. Data in wrong format

cleaned_df_formatted = cleaned_df.convert_dtypes() #convert all the dtypes in the best types for them
cleaned_df_formatted['Price'] = cleaned_df_formatted['Price'].astype(float)
cleaned_df_formatted['Living Area'] = cleaned_df_formatted['Living Area'].astype(float)
cleaned_df_formatted['Number of rooms'] = cleaned_df_formatted['Number of rooms'].astype(float)

#E. Wrong data

index_sell_to_del = cleaned_df_formatted[(cleaned_df_formatted['To sell'] == False)].index # taking the index of the row who are False to delete them on the next line
df_sell = cleaned_df_formatted.drop(index_sell_to_del) #Delete the rows collect the line above and store the new DF in df_sell
del df_sell['To rent'] #Delete the column "To rent" because it is useless now

index_rent_to_del = cleaned_df_formatted[(cleaned_df_formatted['To rent'] == False)].index # taking the index of the row who are False to delete them on the next line
df_rent = cleaned_df_formatted.drop(index_rent_to_del) #Delete the rows collect the line above and store the new DF in df_rent
del df_rent['To sell'] #Delete the column "To sell" because it is useless now

#Delete the extreme prices and extreme living area from df_rent

df_rent.drop(df_rent[df_rent['Price'] >= 4000].index, inplace = True)
df_rent.drop(df_rent[df_rent['Living Area'] > 400].index, inplace = True)

#Creating a Dataframe for renting appartment and Rez-de-chaussee

#Taking only the appartment and Rez-de-chaussée to rent 
df_rent_appart = df_rent[(df_rent.loc[ : ,'type'] == "Appartement") | (df_rent.loc[ : ,'type'] == "Rez-de-chaussée")]

#Calculating the price per squared m for the appartment and Rez-de-chaussée to rent
df_rent_appart.loc[ : ,'Price by squared m'] = df_rent_appart.loc[ : ,'Price'] / df_rent_appart.loc[ : ,'Living Area']

#Calculating the price per bedroom m for the appartment and Rez-de-chaussée to rent
df_rent_appart.loc[ : ,'Price by bedroom'] = df_rent_appart.loc[ : ,'Price'] / df_rent_appart.loc[ : ,'Number of rooms']


#Creating Datafram for selling appartement and Rez-de-chaussée

#Taking only the appartment and Rez-de-chausséet to sell 
df_sell_appart = df_sell[(df_sell.loc[ : ,'type'] == "Appartement") | (df_sell.loc[ : ,'type'] == "Rez-de-chaussée")]
df_sell_appart.drop(df_sell_appart[(df_sell_appart['Price'] < 50000)].index, inplace=True) #Deleting the rows where the price are under 50 000 because some renting appart were true to sell so...

#Calculating the price per squared m for for selling appartement and Rez-de-chaussée
df_sell_appart['Price by squared m'] = df_sell_appart['Price'] / df_sell_appart['Living Area']

#Calculating the price by bedroom for for selling appartement and Rez-de-chaussée
df_sell_appart['Price by bedroom'] = df_sell_appart['Price'] / df_sell_appart['Number of rooms']

#Part 2 : Data Analysis

#How many rows and columns ?

def how_many_columns_rows_df(df):
    vnames = [name for name in globals() if globals()[name] is df]#Getting the name of the df
    print(f"There are {df.shape[0]} rows in our database named {vnames[0]}") #How many rows ?
    print(f"There are {df.shape[1]} columns in our database named {vnames[0]}") #How many columns ?
    return None

#Defining a function to extract a range of zipcode given to this function from one defined dataframe with a give column 
#name (where the zipcode are)

def taking_zip_code_desired_from_df (begin_zipcode, end_zipcode, df, name_of_col_zipcode):

    #Making a bool series
    bool_series = df[name_of_col_zipcode].between(begin_zipcode, end_zipcode, inclusive = True)
    
    #Returning dataframe with zipcode between above values
    df_desired = df[bool_series]

    return df_desired

#Defining the function to calcul the min, max, mean for one province to rent (by default) or to buy

def min_max_mean_df (df, colum_name, province_name, rent = True):  

    min = df[colum_name].min().round(2)
    max = df[colum_name].max().round(2)
    mean = df[colum_name].mean().round(2)

    diction = {province_name : mean}

    if rent == True :
        selling_or_renting = "renting"
    else :
        selling_or_renting = "selling"

    print(f"The minimal {selling_or_renting} {colum_name} in {province_name} Province is {min.round(2)} €/m2")
    print(f"The maximal {selling_or_renting} {colum_name} in {province_name} Province is {max.round(2)} €/m2")
    print(f"The mean {selling_or_renting} {colum_name} in {province_name} Province is {mean.round(2)} €/m2")

    return diction

/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rent_appart.loc[ : ,'Price by squared m'] = df_rent_appart.loc[ : ,'Price'] / df_rent_appart.loc[ : ,'Living Area']
/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rent_appart.loc[ : ,'Price by bedroom'] = df_rent_appart.loc[ : ,'Price'] / df_rent_appart.loc[ : ,'Number of rooms']
/var/folders/b

In [85]:
#Preprocessing Data

#How many rows and columns in my DF ?

how_many_columns_rows_df(df_rent_appart)
how_many_columns_rows_df(df_sell_appart)

#Calculating the correlation of my dataframe(s)

corr = df_rent_appart.corr().abs() #Calculate the correlation between all the variables in the DF and take the absolute value of them

corr['Price']

#Seeing the results of the correlation with the price I decide to delete some columns
#who are not meaningful in my case

print(corr['Price'][corr['Price']<0.36]) #The columns that I'll delete from my df

del df_rent_appart['Unnamed: 0']
del df_rent_appart['Fully equipped kitchen']
del df_rent_appart['Furnished']
del df_rent_appart['Open fire']
del df_rent_appart['Terrace']
del df_rent_appart['Garden']
del df_rent_appart['Swimming pool']
del df_rent_appart['type']
del df_rent_appart['To rent']

There are 8676 rows in our database named df_rent_appart
There are 15 columns in our database named df_rent_appart
There are 15131 rows in our database named df_sell_appart
There are 15 columns in our database named df_sell_appart
Unnamed: 0                0.119638
Fully equipped kitchen    0.069420
Furnished                 0.259539
Open fire                 0.092814
Terrace                   0.189533
Garden                    0.038577
Swimming pool             0.061990
Name: Price, dtype: float64


/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/3343303720.py:10: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_rent_appart.corr().abs() #Calculate the correlation between all the variables in the DF and take the absolute value of them


In [88]:
#Making the df for each province

df_rent_appart_bru = taking_zip_code_desired_from_df(1000, 1299, df_rent_appart ,"zipcode")
df_rent_appart_bw = taking_zip_code_desired_from_df(1300, 1499, df_rent_appart ,"zipcode")
df_rent_appart_anv = taking_zip_code_desired_from_df(2000, 2999, df_rent_appart ,"zipcode")
df_rent_appart_lim = taking_zip_code_desired_from_df(3500, 3999, df_rent_appart ,"zipcode")
df_rent_appart_liege = taking_zip_code_desired_from_df(4000, 4999, df_rent_appart ,"zipcode")
df_rent_appart_nam = taking_zip_code_desired_from_df(5000, 5680, df_rent_appart ,"zipcode")
df_rent_appart_lux = taking_zip_code_desired_from_df(6600, 6999, df_rent_appart ,"zipcode")
df_rent_appart_fl_occi = taking_zip_code_desired_from_df(8000, 8999, df_rent_appart ,"zipcode")
df_rent_appart_fl_occi.drop(df_rent_appart_fl_occi[(df_rent_appart_fl_occi['Price by squared m'] < 1)].index, inplace=True) #Deleting the row where the price by squared m is under 1 because the Price was 45 a month for the appartment there is a mistake I think...
df_rent_appart_fl_ori = taking_zip_code_desired_from_df(9000, 9999, df_rent_appart ,"zipcode")

#For Brabant flamand and Hainaut several range of post code so multiple lines
df_rent_appart_bf1 = taking_zip_code_desired_from_df(1500, 1999, df_rent_appart ,"zipcode")
df_rent_appart_bf2 = taking_zip_code_desired_from_df(3000, 3499, df_rent_appart ,"zipcode")
df_rent_appart_bf = pd.concat([df_rent_appart_bf1, df_rent_appart_bf2], axis = 0)

df_rent_appart_hai1 = taking_zip_code_desired_from_df(6000, 6599, df_rent_appart ,"zipcode")
df_rent_appart_hai2 = taking_zip_code_desired_from_df(7000, 7999, df_rent_appart ,"zipcode")
df_rent_appart_hai = pd.concat([df_rent_appart_hai1, df_rent_appart_hai2], axis = 0)


/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:103: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  bool_series = df[name_of_col_zipcode].between(begin_zipcode, end_zipcode, inclusive = True)
/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:103: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  bool_series = df[name_of_col_zipcode].between(begin_zipcode, end_zipcode, inclusive = True)
/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:103: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  bool_series = df[name_of_col_zipcode].between(begin_zipcode, end_zipcode, inclusive = True)
/var/folders/bf/nzxkpm4s193510wrty2mgpm40000gn/T/ipykernel_17185/4063501412.py:103: FutureWarning: Boolean inputs to the `inclusive` argument are

In [90]:
how_many_columns_rows_df(df_rent_appart_bru)
df_rent_appart_bru.head(10)

There are 1515 rows in our database named df_rent_appart_bru
There are 6 columns in our database named df_rent_appart_bru


,Price,Number of rooms,Living Area,zipcode,Price by squared m,Price by bedroom
30,950.0,1.0,80.0,1050,11.875000,950.0
53,1250.0,1.0,90.0,1050,13.888889,1250.0
67,1200.0,2.0,90.0,1200,13.333333,600.0
68,1200.0,2.0,90.0,1200,13.333333,600.0
80,2500.0,2.0,140.0,1000,17.857143,1250.0
82,2445.0,2.0,120.0,1040,20.375000,1222.5
83,1919.0,1.0,60.0,1000,31.983333,1919.0
84,2245.0,2.0,100.0,1040,22.450000,1122.5
87,2475.0,2.0,100.0,1000,24.750000,1237.5
95,1850.0,1.0,78.0,1000,23.717949,1850.0


In [114]:
#Modelization part for Brussels Province

X = np.array(df_rent_appart_bru.drop(columns = 'Price')) #features
y = np.array(df_rent_appart_bru['Price']) #target

print(f"That's a sample line for the X_features : {X}")

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )

from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X_train, y_train)


training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")

regressor.fit(X_test, y_test)

regressor.predict(X_test)


test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

# define one new instance
#Xnew = [[2.0 100 1000 13 1000]]


Xnew = np.ndarray([2.0, 100, 1000, 13, 1000])
print(type(Xnew))
# make a prediction
ynew = regressor.predict(Xnew)
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))

type(X_test)


That's a sample line for the X_features : [[1.0 80.0 1050 11.875 950.0]
 [1.0 90.0 1050 13.88888888888889 1250.0]
 [2.0 90.0 1200 13.333333333333334 600.0]
 ...
 [1.0 44.0 1050 21.59090909090909 950.0]
 [1.0 68.0 1040 12.941176470588236 880.0]
 [1.0 50.0 1040 16.0 800.0]]
The score of the model for training part : 0.9313653249747909
The score of the model for testing part : 0.9349016264546737


TypeError: 'float' object cannot be interpreted as an integer